# This Notebook uses the IndicNER model which i have already Trained and fine_tuned on Naampadam (marathi) dataset.
### (training and finetuning of this model is done by another notebook named 'training_indicNER')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate -U
!pip install transformers datasets sentencepiece tokenizers segeval -q
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e1cc7801425c4360feabe1615d445a296a11440225a38183a1e482d9c97e1680
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:

from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import accelerate
import torch
import numpy as np

In [4]:
import os
import json

In [5]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/checkpoint-31250') #This path is different than path I used while saving checkpoints. This is because, my colab gpu limit exceeded, so i just copied that checkpoint in my another google account drive
model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/checkpoint-31250')
data_collator = DataCollatorForTokenClassification(tokenizer)

In [6]:
marathi_dataset = load_dataset('ai4bharat/naamapadam', 'mr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/455248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1080 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2300 [00:00<?, ? examples/s]

In [7]:
label_list= marathi_dataset['train'].features['ner_tags'].feature.names
label_list
model.config.id2label ={0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}

In [8]:
def tokenize_and_align_label(example, label_all_tokens=True):  #input is example from dataset and a boolean argument
  tokenized_input= tokenizer(example['tokens'], truncation=True, is_split_into_words=True)      #tokenizing the input
  #print("Before aligning the labels:",tokenized_input)
  #e.g. ['[CLS]', 'वि', '##दय', '##ाप', '##ी', '##ठ', '/', 'राजय', 'सरकार', '/', 'कदर', 'सरकार', '/', 'सबध', '##ित', 'ससथा', '##च', 'न', '##ियम', '[SEP]']

  labels= []   #This list will contain correct labels for tokenized input of the example
  for i, label in enumerate(example['ner_tags']):
    word_ids = tokenized_input.word_ids(batch_index=i)    #Extracting word id of tokenized input with batch_index i
    #here word_ids() function call will return list of mapping the tokens to their actual word in initial sentence.
    #It returns the list indicating the word corresponding to each token. For example mentioned in comment above word_ids list is [None, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, None]

    previous_word_idx =None
    label_ids=[]              #to set labels for tokenized input
    for word_idx in word_ids:
      if word_idx is None:    #then just setting it as -100 for special tokens
        label_ids.append(-100)
      elif word_idx != previous_word_idx:   #it means this token has came from new word. And it has its own label so just add label of that word
        label_ids.append(label[word_idx])
      else:                   #
        #to take care of sub_words which has same word_idx
        #So I am just setting those also as -100. But only if label_all_tokens==False
        label_ids.append(label[word_idx] if label_all_tokens else -100)
      previous_word_idx= word_idx

    labels.append(label_ids)      #after completing inner loop just appending labels_ids to overall labels

  #Now all labels are calculated. So I am just inserting new 'labels' into tokenized_input. It already had 'input_ids','attension_mask', 'token_type_ids' as keys.
  #I have printed tokenized input of one of the example in somewhere above cells
  tokenized_input['labels'] = labels
  return tokenized_input



In [9]:
metric=load_metric("seqeval")
from seqeval.metrics import classification_report

from sklearn.metrics import f1_score, precision_recall_fscore_support
def compute_metrics(eval_preds):
  pred_logits, labels =eval_preds
  pred_logits = np.argmax(pred_logits,axis=2) #logits and probabilities are in the same order, Hence I am not applying the softmax
  true_predictions = [
      [label_list[eval_preds] for (eval_preds,l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
   ]  #as I need to pass a list in prediction argument in metric.compute
  true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
  results=metric.compute(predictions=true_predictions, references=true_labels)

  #For calculation of macro F1 score, Calculate precision, recall, and f1 scores for each label
  label_precision, label_recall, label_f1, _ = precision_recall_fscore_support(np.concatenate(true_labels), np.concatenate(true_predictions), average=None, labels=label_list)
  macro_f1 = np.mean(label_f1)

  return {
      "precision" : results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
      "macro_f1": macro_f1,
      "labelwise_f1":label_f1,
  }

<ipython-input-9-17a43fb75357>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [10]:
# both true_outputs and predicted_outputs are list of list. i.e. first element of true_outputs will be a list of true_labels for first sentence. similarly for predicted_outputs
from sklearn.metrics import precision_recall_fscore_support
def Calculate_performance(true_outputs,predicted_outputs):      #function to calculate classwise precision, recall, f1_score

  # Flatten the lists for calculation
  flat_true_labels = [label for sublist in true_outputs for label in sublist]
  flat_predicted_labels = [label for sublist in predicted_outputs for label in sublist]

  # Calculate precision, recall, and F1-score
  classwise_precision, classwise_recall, classwise_f1_score, _ = precision_recall_fscore_support(flat_true_labels, flat_predicted_labels, average=None, labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC'],zero_division=1)
  return (classwise_precision,classwise_recall,classwise_f1_score)

In [11]:
def get_prections_on_sentences(all_sentences,all_correct_labels):       #Function to get prediction on sentences. here sentences is a list of list.
  for i in range(len(all_sentences)):
    if(len(all_sentences[i])!=len(all_correct_labels[i])):
      print("Unequal lengths of sentence and labels at index i=",i," and Lengths are ",len(all_sentences[i]),", ",len(all_correct_labels[i]))
  all_predicted_labels=[]                           #this list will be returned from function
  unequal_length_labels_indices=[]
  for i in range(len(all_sentences)):
    sentence= ' '.join(all_sentences[i])
    predicted_labels = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model
                                    )
    if(len(predicted_labels) != len(all_correct_labels[i])):      #if predicted label length is not equal to actual_label length then there is a problem
      #print(i,predicted_labels, all_correct_labels[i])
      unequal_length_labels_indices.append(i)                             #storing the index of that sentence,so that it won't be a problem while predicting.
    all_predicted_labels.append(predicted_labels)
  return (all_predicted_labels,unequal_length_labels_indices)



In [12]:
#This is the function to get predictions on random sentence
def get_predictions( sentence, tokenizer, model ):
  tok_sentence = tokenizer(sentence, return_tensors='pt')   #performing tokenization on the sentence

  with torch.no_grad():
    logits = model(**tok_sentence).logits.argmax(-1)  #I am sending the tokenized sentence to the model to get predictions
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]] #mapping the maximum predicted class id with the class label

    predicted_labels = []

    previous_token_id = 0

    #As we need to assign the named entity label to the head word(i.e. Actual word) and not its sub-words. Below code does this this thing
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [13]:
args= TrainingArguments(
    output_dir="/content/drive/MyDrive/Mtech/trained_model/ner_model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    logging_dir="/content/drive/MyDrive/Mtech/loggingDirectory/logs",
    logging_strategy="steps",
    logging_steps=2000,  # Log every 1000 steps
    save_strategy="epoch",
    save_steps=2000,
)
train_dataset = marathi_dataset["train"].select(range(100000))
train_dataset = train_dataset.map(
    tokenize_and_align_label,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
eval_dataset = marathi_dataset["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_label,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

test_dataset = marathi_dataset["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_label,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)


Running tokenizer on train dataset (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

Running tokenizer on Validation dataset (num_proc=4):   0%|          | 0/2300 [00:00<?, ? examples/s]

Running tokenizer on test dataset (num_proc=4):   0%|          | 0/1080 [00:00<?, ? examples/s]

In [14]:

index_to_label={0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
text_column_name = "tokens"
label_column_name = "ner_tags"
column_names = marathi_dataset["train"].column_names
features = marathi_dataset["train"].features
label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

## Question 2 answer


##### Method 1: (I calculated required performance parameters by two seperate methods. You can use any of the method)

###### Predicting on test Dataset from marathi corpus

In [15]:
#To get macro f1 score(and other performance measures) on test dataset
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset)

In [16]:
#Performance of fine tuned model on test dataset
print("All performance Measures for test_dataset: ",test_metrics)
print("precision for test_dataset: ",test_metrics['test_precision'])
print("recall for test_dataset: ",test_metrics['test_recall'])
print("\nLabelwise Test F1 scores\n")
for i in range(len(test_metrics['test_labelwise_f1'])):
  print(label_list[i]," \t ",test_metrics['test_labelwise_f1'][i])
print("Macro f1 score on test dataset: ",test_metrics["test_macro_f1"])

All performance Measures for test_dataset:  {'test_loss': 0.36820700764656067, 'test_precision': 0.8265765765765766, 'test_recall': 0.7867862015201715, 'test_f1': 0.8061907139291064, 'test_accuracy': 0.9156863472742791, 'test_macro_f1': 0.7953584316630105, 'test_labelwise_f1': array([0.95484104, 0.88456805, 0.91235195, 0.74187231, 0.59528131,
       0.84191617, 0.6366782 ]), 'test_runtime': 10.3568, 'test_samples_per_second': 104.28, 'test_steps_per_second': 6.566}
precision for test_dataset:  0.8265765765765766
recall for test_dataset:  0.7867862015201715

Labelwise Test F1 scores

O  	  0.9548410404624277
B-PER  	  0.8845680531626876
I-PER  	  0.9123519458544839
B-ORG  	  0.7418723070896983
I-ORG  	  0.5952813067150635
B-LOC  	  0.8419161676646706
I-LOC  	  0.6366782006920415
Macro f1 score on test dataset:  0.7953584316630105


In [17]:
test_results= {
    "class_labels": label_list,
    "classwise_f1_score": test_metrics['test_labelwise_f1'].tolist(),
    "macro_f1_score": test_metrics["test_macro_f1"],
    "recall": test_metrics['test_recall'],
    "precision": test_metrics['test_precision']
}
os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/', exist_ok=True)
with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/test_results.json', 'w') as outfile:
    json.dump(test_results, outfile, indent=4)

###### Predicting on validation Dataset from marathi corpus

In [18]:
#to get macro f1 score(and other performance measures) on validation dataset
eval_results = trainer.evaluate()

Trainer is attempting to log a value of "[0.95563104 0.86982032 0.88208092 0.7619221  0.71291742 0.83151621
 0.65938567]" of type <class 'numpy.ndarray'> for key "eval/labelwise_f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [19]:
#Performance of fine tuned model on validation dataset
print("All performance Measures for validation_dataset: ",eval_results)
print("precision for validation_dataset: ",eval_results['eval_precision'])
print("recall for validation_dataset: ",eval_results['eval_recall'])
print("\nLabelwise validation F1 scores\n")
for i in range(len(eval_results['eval_labelwise_f1'])):
  print(label_list[i]," \t ",eval_results['eval_labelwise_f1'][i])
print("Macro f1 score on validation dataset: ",eval_results["eval_macro_f1"])

All performance Measures for validation_dataset:  {'eval_loss': 0.3486127555370331, 'eval_precision': 0.8021912688975709, 'eval_recall': 0.8145752479517033, 'eval_f1': 0.8083358295177372, 'eval_accuracy': 0.9188956387705141, 'eval_macro_f1': 0.8104676676314676, 'eval_labelwise_f1': array([0.95563104, 0.86982032, 0.88208092, 0.7619221 , 0.71291742,
       0.83151621, 0.65938567]), 'eval_runtime': 11.9916, 'eval_samples_per_second': 191.801, 'eval_steps_per_second': 12.008}
precision for validation_dataset:  0.8021912688975709
recall for validation_dataset:  0.8145752479517033

Labelwise validation F1 scores

O  	  0.9556310403937365
B-PER  	  0.8698203153648698
I-PER  	  0.8820809248554914
B-ORG  	  0.7619220968329087
I-ORG  	  0.7129174216929848
B-LOC  	  0.8315162087512721
I-LOC  	  0.6593856655290102
Macro f1 score on validation dataset:  0.8104676676314676


In [20]:
val_results= {
    "class_labels": label_list,
    "classwise_f1_score": eval_results['eval_labelwise_f1'].tolist(),
    "macro_f1_score": test_metrics["test_macro_f1"],
    "recall": eval_results["eval_macro_f1"],
    "precision": eval_results['eval_precision']
}
os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/', exist_ok=True)
with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/val_results.json', 'w') as outfile:
    json.dump(val_results, outfile, indent=4)

###### Predicting on training Dataset from marathi corpus

In [21]:
#To get macro f1 score on train dataset
train_predictions, train_labels, train_metrics = trainer.predict(train_dataset)

In [22]:
#Performance of fine tuned model on train dataset
print("All performance Measures for train_dataset: ",train_metrics)
print("precision for train_dataset: ",train_metrics['test_precision'])
print("recall for train_dataset: ",train_metrics['test_recall'])
print("\nLabelwise train F1 scores\n")
for i in range(len(train_metrics['test_labelwise_f1'])):
  print(label_list[i]," \t ",train_metrics['test_labelwise_f1'][i])
print("Macro f1 score on train dataset: ",train_metrics["test_macro_f1"])

All performance Measures for train_dataset:  {'test_loss': 0.058550961315631866, 'test_precision': 0.940665824025164, 'test_recall': 0.9434294550852584, 'test_f1': 0.9420456126787785, 'test_accuracy': 0.9797794604373435, 'test_macro_f1': 0.9528096471875028, 'test_labelwise_f1': array([0.98922316, 0.96402628, 0.96428388, 0.94114513, 0.94963803,
       0.95563365, 0.9057174 ]), 'test_runtime': 567.4449, 'test_samples_per_second': 176.229, 'test_steps_per_second': 11.014}
precision for train_dataset:  0.940665824025164
recall for train_dataset:  0.9434294550852584

Labelwise train F1 scores

O  	  0.9892231627328928
B-PER  	  0.9640262814694017
I-PER  	  0.9642838830238772
B-ORG  	  0.9411451328361077
I-ORG  	  0.9496380269559347
B-LOC  	  0.9556336473983533
I-LOC  	  0.9057173958959529
Macro f1 score on train dataset:  0.9528096471875028


In [23]:
train_results= {
    "class_labels": label_list,
    "classwise_f1_score": train_metrics['test_labelwise_f1'].tolist(),
    "macro_f1_score": train_metrics["test_macro_f1"],
    "recall": train_metrics['test_recall'],
    "precision": train_metrics['test_precision']
}
os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/', exist_ok=True)
with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/Automatic_results/train_results.json', 'w') as outfile:
    json.dump(train_results, outfile, indent=4)

##### Method2 : Its is also another method to get performance parameters but I didn't executed this because of my low computational resources
  (I tried with smaller dataset and it worked correctly. But for  naampadam dataset it was taking too much time )

###### On test data

In [29]:
# test_sentences=marathi_dataset["test"]["tokens"]
# correct_test_labels=marathi_dataset["test"]["ner_tags"]
# correct_test_labels = [[index_to_label[index] for index in sublist] for sublist in correct_test_labels] #as dataset has label ids not actual labels. Hence changing ner_ids to ner_tags

In [30]:
# test_predicted_labels,test_unequal_length_indices=get_prections_on_sentences(test_sentences,correct_test_labels)

In [31]:
# #Handling the sentences which have uneuqal length of labels for correct label and predicted labels
# test_predicted_labels = [element for index, element in enumerate(test_predicted_labels) if index not in test_unequal_length_indices]
# correct_test_labels = [element for index, element in enumerate(correct_test_labels) if index not in test_unequal_length_indices]
# len(test_predicted_labels)

In [32]:
# classwise_precision, classwise_recall, classwise_f1_score=Calculate_performance(correct_test_labels,test_predicted_labels)
# precision = np.mean(classwise_precision)
# recall = np.mean(classwise_recall)
# labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC']
# print("class_labels: ",labels)
# # print("\nclasswise_precision: ",classwise_precision)
# # print("\nclasswise_recall: ",classwise_recall)
# print("\nclasswise_f1_score: ",classwise_f1_score)
# macro_f1_score=np.mean(classwise_f1_score)
# print("\nmacro_f1_score: ",macro_f1_score)
# print("precision: ",precision)
# print("recall: ",recall)

In [33]:
# test_results= {
#     "class_labels": labels,
#     "classwise_f1_score": classwise_f1_score.tolist(),
#     "macro_f1_score": macro_f1_score,
#     "recall": recall,
#     "precision": precision
# }
# os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/', exist_ok=True)
# with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/test_results.json', 'w') as outfile:
#     json.dump(test_results, outfile, indent=4)

###### On validation Data

In [34]:
# validation_sentences=marathi_dataset["validation"]["tokens"]
# correct_val_labels=marathi_dataset["validation"]["ner_tags"]
# correct_val_labels = [[index_to_label[index] for index in sublist] for sublist in correct_val_labels] #as dataset has label ids not actual labels. Hence changing ner_ids to ner_tags

In [35]:
# val_predicted_labels,val_unequal_length_indices=get_prections_on_sentences(validation_sentences,correct_val_labels)

In [36]:
# #Handling the sentences which have uneuqal length of labels for correct label and predicted labels
# val_predicted_labels = [element for index, element in enumerate(val_predicted_labels) if index not in val_unequal_length_indices]
# correct_val_labels = [element for index, element in enumerate(correct_val_labels) if index not in val_unequal_length_indices]
# len(val_predicted_labels)

In [37]:
# classwise_precision, classwise_recall, classwise_f1_score=Calculate_performance(correct_val_labels,val_predicted_labels)
# precision = np.mean(classwise_precision)
# recall = np.mean(classwise_recall)
# labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC']
# print("class_labels: ",labels)
# # print("\nclasswise_precision: ",classwise_precision)
# # print("\nclasswise_recall: ",classwise_recall)
# print("\nclasswise_f1_score: ",classwise_f1_score)
# macro_f1_score=np.mean(classwise_f1_score)
# print("\nmacro_f1_score: ",macro_f1_score)
# print("precision: ",precision)
# print("recall: ",recall)

In [38]:
# val_results= {
#     "class_labels": labels,
#     "classwise_f1_score": classwise_f1_score.tolist(),
#     "macro_f1_score": macro_f1_score,
#     "recall": recall,
#     "precision": precision
# }
# os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/', exist_ok=True)
# with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/val_results.json', 'w') as outfile:
#     json.dump(val_results, outfile, indent=4)

###### On train dataset

In [39]:
# train_sentences=marathi_dataset["train"]["tokens"]
# correct_train_labels=marathi_dataset["train"]["ner_tags"]
# correct_train_labels = [[index_to_label[index] for index in sublist] for sublist in correct_train_labels]

In [40]:
# train_predicted_labels,train_unequal_length_indices=get_prections_on_sentences(train_sentences,correct_train_labels)

In [41]:
# #Handling the sentences which have uneuqal length of labels for correct label and predicted labels
# train_predicted_labels = [element for index, element in enumerate(train_predicted_labels) if index not in train_unequal_length_indices]
# correct_train_labels = [element for index, element in enumerate(correct_train_labels) if index not in train_unequal_length_indices]
# len(train_predicted_labels)

In [42]:
# classwise_precision, classwise_recall, classwise_f1_score=Calculate_performance(correct_train_labels,train_predicted_labels)
# precision = np.mean(classwise_precision)
# recall = np.mean(classwise_recall)
# labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC']
# print("class_labels: ",labels)
# # print("\nclasswise_precision: ",classwise_precision)
# # print("\nclasswise_recall: ",classwise_recall)
# print("\nclasswise_f1_score: ",classwise_f1_score)
# macro_f1_score=np.mean(classwise_f1_score)
# print("\nmacro_f1_score: ",macro_f1_score)
# print("precision: ",precision)
# print("recall: ",recall)

In [43]:
# train_results= {
#     "class_labels": labels,
#     "classwise_f1_score": classwise_f1_score.tolist(),
#     "macro_f1_score": macro_f1_score,
#     "recall": recall,
#     "precision": precision
# }
# os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/', exist_ok=True)
# with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que2/NER_Results/manual_results/train_results.json', 'w') as outfile:
#     json.dump(train_results, outfile, indent=4)

###### Just to show prediction output on one of the sentence

In [60]:
test_sentences=marathi_dataset["test"]["tokens"][0:10]
correct_test_labels=marathi_dataset["test"]["ner_tags"][0:10]
correct_test_labels = [[index_to_label[index] for index in sublist] for sublist in correct_test_labels] #as dataset has label ids not actual labels. Hence changing ner_ids to ner_tags
test_predicted_labels,test_unequal_length_indices=get_prections_on_sentences(test_sentences,correct_test_labels)
#Handling the sentences which have uneuqal length of labels for correct label and predicted labels
test_predicted_labels = [element for index, element in enumerate(test_predicted_labels) if index not in test_unequal_length_indices]
correct_test_labels = [element for index, element in enumerate(correct_test_labels) if index not in test_unequal_length_indices]
len(test_predicted_labels)
print("One of the sentence from test dataset: ",test_dataset[2]['tokens'])
print(test_predicted_labels[2])
print(correct_test_labels[2])


One of the sentence from test dataset:  ['राष्ट्रवादीचे', 'सर्वेसर्वा', 'व', 'माजी', 'केंद्रीय', 'कृषीमंत्री', 'शरद', 'पवार', 'अखेर', 'आ', '.']
['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O']
['B-ORG', 'O', 'O', 'O', 'B-ORG', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O']


## If want to predict in any random sentence, then execute below lines

In [15]:
#This is the function to get predictions on random sentence
def get_predictions( sentence, tokenizer, model ):
  tok_sentence = tokenizer(sentence, return_tensors='pt')   #performing tokenization on the sentence

  with torch.no_grad():
    logits = model(**tok_sentence).logits.argmax(-1)  #I am sending the tokenized sentence to the model to get predictions
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]] #mapping the maximum predicted class id with the class label

    predicted_labels = []

    previous_token_id = 0

    #As we need to assign the named entity label to the head word(i.e. Actual word) and not its sub-words. Below code does this this thing
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [16]:
punctuation_marks = ['.', ',', '?', '!', ';', ':', '-', '(', ')', '[', ']',
                     '{', '}', '...', '"', "'", '/', '\\', '&', '%', '$',
                     '#', '@', '*', '+', '=', '<', '>', '~','’','‘']
def split_sentence(sentence):       #splitting the sentence in words. Also making sure all punctuation marks are seperate elements because out tokenizer is doing such way while predicting tokens
    words = []
    current_word = ''
    for char in sentence:
        if((char not in punctuation_marks) and char != " "):
            current_word += char
        else:
            if current_word:
                words.append(current_word)
                current_word = ''
            words.append(char)
    if current_word:
        words.append(current_word)
    return words

###### If want to see prediction on any single sentence then execute below cell

In [17]:
sentence = ' '.join(['यावर्षी', "'", 'गुरू', "'", 'चित्रपटामुळे', 'तिला', 'यश', 'मिळाले', 'तर', 'प्रोव्होक्ड', "'", 'मुळे', 'तिच्या', 'अभिनयाची', 'प्रशंसा', 'झाली', '.'])
# sentence = ' '.join(['कार्यकर्त्यांमधील', 'वाद', 'बघून', 'संतापलेल्या', 'सुप्रिया', 'सुळे', 'यांनी', 'कार्यक्रमामध्येच', 'राडेबाज', 'कार्यकर्त्यांना', 'दम', 'दिला', '.'])
predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )
words_list = split_sentence(sentence)
filtered_list = [s.strip() for s in words_list if s.strip() != ""]      #removing all whitespace characters from it
# print(len(filtered_list))
# print(len(predicted_labels))
for index in range(len(filtered_list)):
  print( filtered_list[index] + '\t' + predicted_labels[index] )

यावर्षी	O
'	O
गुरू	B-PER
'	O
चित्रपटामुळे	O
तिला	O
यश	O
मिळाले	O
तर	O
प्रोव्होक्ड	O
'	O
मुळे	O
तिच्या	O
अभिनयाची	O
प्रशंसा	O
झाली	O
.	O


## Solution for Question4

In [18]:
# both true_outputs and predicted_outputs are list of list. i.e. first element of true_outputs will be a list of true_labels for first sentence. similarly for predicted_outputs
from sklearn.metrics import precision_recall_fscore_support
def Calculate_performance(true_outputs,predicted_outputs):      #function to calculate classwise precision, recall, f1_score

  # Flatten the lists for calculation
  flat_true_labels = [label for sublist in true_outputs for label in sublist]
  flat_predicted_labels = [label for sublist in predicted_outputs for label in sublist]

  # Calculate precision, recall, and F1-score
  classwise_precision, classwise_recall, classwise_f1_score, _ = precision_recall_fscore_support(flat_true_labels, flat_predicted_labels, average=None, labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'],zero_division=1)
  return (classwise_precision,classwise_recall,classwise_f1_score)

In [19]:
def get_prections_on_sentences(all_sentences,all_correct_labels):       #Function to get prediction on sentences. here sentences is a list of list.
  for i in range(len(all_sentences)):
    if(len(all_sentences[i])!=len(all_correct_labels[i])):
      print("Unequal lengths of sentence and labels at index i=",i," and Lengths are ",len(all_sentences[i]),", ",len(all_correct_labels[i]))
  all_predicted_labels=[]                           #this list will be returned from function
  unequal_length_labels_indices=[]
  for i in range(len(all_sentences)):
    sentence= ' '.join(all_sentences[i])
    predicted_labels = get_predictions(sentence=sentence,
                                    tokenizer=tokenizer,
                                    model=model
                                    )
    if(len(predicted_labels) != len(all_correct_labels[i])):      #if predicted label length is not equal to actual_label length then there is a problem
      #print(i,predicted_labels, all_correct_labels[i])
      unequal_length_labels_indices.append(i)                             #storing the index of that sentence,so that it won't be a problem while predicting.
    all_predicted_labels.append(predicted_labels)
  return (all_predicted_labels,unequal_length_labels_indices)



In [20]:

sentences25= [
                ['मला', 'तर', 'हे', 'आताच', 'समजतेय', ',', 'असं', 'म्हणून', 'तिने', 'या', 'चर्चापासून', 'लांबच', 'राहण्याचा', 'प्रयत्न', 'केला', '.'],
                ['धातू', 'काठी', 'आणि', 'लॉक', 'आपण', 'कव्हर', 'बंद', 'करण्यास', 'अनुमती', 'देते'],
                ['श्रीराम', 'लागू', '-', 'मराठी', 'रंगभूमीचे', 'अनभिषिक्त', 'सम्राट', ',', 'चतुरस्र', 'अभिनेते', ',', 'परखड', '.', '.', '30', 'डिसेंबरला', 'होणाऱ्या', 'शपथविधी', 'सोहळ्याची', 'विधानभवनात', 'जोरदार', 'तयारी', 'सुरू'],
                ['यावर्षी', "'", 'गुरू', "'", 'चित्रपटामुळे', 'तिला', 'यश', 'मिळाले', 'तर', 'प्रोव्होक्ड', "'", 'मुळे', 'तिच्या', 'अभिनयाची', 'प्रशंसा', 'झाली', '.'],
                ['कार्यकर्त्यांमधील', 'वाद', 'बघून', 'संतापलेल्या', 'सुप्रिया', 'सुळे', 'यांनी', 'कार्यक्रमामध्येच', 'राडेबाज', 'कार्यकर्त्यांना', 'दम', 'दिला', '.'],
                ['‘', 'माइन', 'काम्फ', '’', 'मध्ये', 'हिटलर', 'सांगतो', '.', '.', 'हे', 'एक', 'स्वयंप्रकाशी', 'सत्य', 'आहे', ',', 'की', 'महाअसत्यामध्ये', 'नेहमीच', 'एक', 'जोरकस', 'विश्वासार्हता', 'असते', '.'],
                ['धानोरी', 'येथील', 'बाबूराव', 'टिंगरे', 'शाळेतील', 'ईव्हीएम', 'मशीनमध्ये', '.', '.', '.', 'पावसाची', 'विश्रांती', ';', 'मात्र', 'मतदाराचा', 'प्रतिसाद', 'कमी'],
                ['भारतीय', 'रेल्वेच्या', '166', 'वर्षांच्या', 'प्रदीर्घ', 'इतिहासामध्ये', 'यंदा', 'कोणत्याही', '.', '.', '.', 'कॅगचा', 'अहवालाचा', 'सुलभ', 'भाषेत', 'अर्थ', 'आहे', ',', 'रेल्वे', '98', 'रुपये', '44', 'पैसे', 'लावून', 'केवळ', '100', 'रुपये', 'कमावत', 'आहे'],
                ['बॅग', 'दागिन्यांसह', 'पुन्हा', 'मिळाल्याने', 'चौहान', 'कुटुंबीयांस', 'मकरसंक्रातीच्या', 'दिवशी', 'सुखद', 'गोड', 'धक्का', 'बसला', 'आहे', '.'],
                ['दरम्यान', ',', 'याविषयी', 'मुकुंदवाडी', 'पोलिस', 'ठाण्यात', 'बसचालकाविरोधात', 'गुन्हा', 'नोंदविण्यात', 'आला', 'आहे', '.'],
                ['श्रींची', 'पालखी', 'शेगावात', 'दाखल', ';', 'खामगाव', 'ते', 'शेगाव', 'रस्ता', 'भक्तांच्या', 'गर्दीने', 'फुलला'],
                ['चंदगड', 'विधानसभा', 'मतदारसंघातून', 'त्यांनी', 'लढावे', ',', 'अशी', 'तेथील', 'भाजपा', 'नेत्यांची', 'इच्छा', 'आहे', '.'],
                ['त्यामुळे', 'पश्चिम', 'रेल्वेवरील', 'लोकसेवा', '१०', 'ते', '१५', 'मिनिटं', 'उशिराने', 'धावतील', '.'],
                ['मात्र', ',', 'प्रत्यक्षात', 'हवामान', 'बदल', 'असो', 'की', ',', 'जागतिक', 'शांततेचा', 'विषय', 'असो', 'अमेरिका', 'आणि', 'इतर', 'विकसित', 'देश', 'विकसनशील', 'देशांवर', 'आपली', 'मते', 'लादताना', 'दिसतात', '.'],
                ['नेपाळच्या', 'सौनाली', 'जवळील', 'पडासरी', 'भागातील', 'एका', 'कारखान्यात', 'झालेल्या', 'प्रचंड', 'स्फोटांच्या', 'झळा', 'भारतीय', 'सीमेपर्यंतही', 'पोहोचल्या', 'असून', 'या', 'भागात', 'सीमा', 'सुरक्षा', 'दलाची', 'गस्त', 'वाढविली', 'गेली', 'आहे', '.'],
                ['लोकसभा', 'निवडणुकीपूर्वी', 'राम', 'मंदिर', 'बांधणारच', 'असं', 'सांगत', 'मतं', 'मिळवणारं', 'मोदी', 'सरकार', 'आता', 'मंदिर', 'केव्हा', 'बांधणार', 'हे', 'सांगायलाच', 'तयार', 'नाहीये', '.'],
                ['जेव्हा', 'उदयनराजे', 'गाणं', 'गातात', '.', '.', '.', 'साताऱ्यामध्ये', 'आज', 'उत्कृष्ट', 'पत्रकार', 'सन्मान', 'पुरस्काराच्या', 'कार्यक्रमात', 'खासदार', 'उदयनराजे', 'यांनी', 'गाणं', 'म्हटल्यानं', 'अनेकांना', 'आश्चर्याचा', 'धक्का', 'बसला', '.'],
                ['माजी', 'जागतिक', 'उपविजेता', 'मुंबईचा', 'दुसरा', 'मानांकित', 'मोहम्मद', 'गुफरानने', 'मुबंइ', 'उपनगरच्या', 'इश्तियाक', 'अन्सारीचे', '25-6', ',', '25-8', 'असे', 'आव्हान', 'परतवून', 'लावले', '.'],
                ['हा', 'प्रकल्प', 'साकारताना', 'समुद्रात', 'टाकण्यात', 'येणार्‍या', 'भरावामुळे', 'भरतीच्या', 'वेळी', 'राहत्या', 'घराचा', 'प्रश्नसुद्धा', 'उभा', 'राहणारा', 'आहे', '.'],
                ['काँग्रेस', '३१', 'जागा', 'मिळवून', 'तिस-या', 'क्रमांकावर', ',', 'तर', 'राष्ट्रवादी', 'काँग्रेसला', 'दोन', 'आकडी', 'संख्याही', 'गाठता', 'आली', 'नाही', '.'],
                ['यामध्ये', 'करणवीर', 'बोहरा', ',', 'दीपिका', 'कक्कर', ',', 'श्रीशांत', 'या', 'सेलिब्रेटींचा', 'समावेश', 'आहे', '.'],
                ['गेले', 'महिनाभर', 'लग्नाचा', 'प्रचार', 'केला', 'आणि', 'महा', '(', 'ग', ')', 'एपिसोडच्या', 'नावाखाली', 'डोंगर', 'पोखरुन', 'उंदीर', 'काढला', '.'],
                ['कार्यक्रम', 'सर्वांना', 'खुला', 'असून', ',', 'रसिकांनी', 'उपस्थित', 'रहावे', 'असे', 'आवाहन', 'लोकमान्य', 'ग्रंथालयाने', 'केले', 'आहे', '.'],
                ['वसई', 'विरार', 'शहर', 'महानगरपालिकेच्या', 'महापौर', 'प्रविणा', 'ठाकूर', 'यांनी', 'पत्रकार', 'परिषदेत', 'स्पर्धेच्या', 'सातव्या', 'पर्वाची', 'घोषणा', 'केली', '.'],
                ['अर्थात', 'यात', 'काही', 'चांगल्या', 'आणि', 'व्हिडिओला', 'पाठिंबा', 'देणाऱ्या', 'कमेंटही', 'होत्या', '.']



        ]

sentences25_correct_labels=[
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','B-MISC', 'I-MISC', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O' ],
    ['O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-MISC', 'I-MISC', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-PER', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O','O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O'],
    ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
]


###### Calculating performance of predictions of model by considering manually solved labels from que 1 as truth labels

In [61]:
sentences25_predicted_labels,unequal_length_indices=get_prections_on_sentences(sentences25,sentences25_correct_labels)

In [22]:
#Handling the sentences which have uneuqal length of labels for correct label and predicted labels
sentences25_predicted_labels = [element for index, element in enumerate(sentences25_predicted_labels) if index not in unequal_length_indices]
sentences25_correct_labels = [element for index, element in enumerate(sentences25_correct_labels) if index not in unequal_length_indices]
#len(sentences25_predicted_labels)


In [23]:
classwise_precision, classwise_recall, classwise_f1_score=Calculate_performance(sentences25_correct_labels,sentences25_predicted_labels)
precision = np.mean(classwise_precision)
recall = np.mean(classwise_recall)
labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
print("class_labels: ",labels)
# print("\nclasswise_precision: ",classwise_precision)
# print("\nclasswise_recall: ",classwise_recall)
print("\nclasswise_f1_score: ",classwise_f1_score)
macro_f1_score=np.mean(classwise_f1_score)
print("\nmacro_f1_score: ",macro_f1_score)
print("precision: ",precision)
print("recall: ",recall)

class_labels:  ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

classwise_f1_score:  [0.96449704 0.75       0.90909091 0.66666667 0.57142857 0.76923077
 0.         0.         0.        ]

macro_f1_score:  0.5145459953152262
precision:  0.8958113943787297
recall:  0.5044344487769564


In [24]:
sentences25_predicted_results= {
    "class_labels": labels,
    "classwise_f1_score": classwise_f1_score.tolist(),
    "macro_f1_score": macro_f1_score,
    "recall": recall,
    "precision": precision
}
os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que4/indicNER/', exist_ok=True)
with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que4/indicNER/sentences25_predicted_results.json', 'w') as outfile:
    json.dump(sentences25_predicted_results, outfile, indent=4)

###### Calculating performance of chatgpt by considering manually solved labels from que 1 as truth labels

In [25]:
sentences25_chatgpt_labels=[
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O'],
    ['B-LOC', 'I-LOC', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

]
sentences25_correct_labels=[
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','B-MISC', 'I-MISC', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O' ],
    ['O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-MISC', 'I-MISC', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'B-PER', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-PER', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O','O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O'],
    ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O','O'],
    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
]


In [26]:
classwise_precision, classwise_recall, classwise_f1_score=Calculate_performance(sentences25_correct_labels,sentences25_chatgpt_labels)
precision = np.mean(classwise_precision)
recall = np.mean(classwise_recall)
labels=['O','B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
print("class_labels: ",labels)
# print("\nclasswise_precision: ",classwise_precision)
# print("\nclasswise_recall: ",classwise_recall)
print("\nclasswise_f1_score: ",classwise_f1_score)
macro_f1_score=np.mean(classwise_f1_score)
print("\nmacro_f1_score: ",macro_f1_score)
print("precision: ",precision)
print("recall: ",recall)

class_labels:  ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

classwise_f1_score:  [0.94354839 0.81481481 0.8        0.625      0.42857143 0.31578947
 0.2        0.         0.        ]

macro_f1_score:  0.45863601157413647
precision:  0.6317460317460317
recall:  0.41634387269980494


In [27]:
sentences25_chatgpt_results= {
    "class_labels": labels,
    "classwise_f1_score": classwise_f1_score.tolist(),
    "macro_f1_score": macro_f1_score,
    "recall": recall,
    "precision": precision
}
os.makedirs('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que4/indicNER/', exist_ok=True)
with open('/content/drive/MyDrive/Mtech2ndSEM/CS689ComputationalLinguistic/FinalAssign2/IndicNER/results/Que4/indicNER/sentences25_chatgpt_results.json', 'w') as outfile:
    json.dump(sentences25_chatgpt_results, outfile, indent=4)

In [28]:
print("All cells executed")

All cells executed
